In [1]:
from flask_login import current_user
import visual_library_plos as vl
import simplejson
from urllib.parse import urlencode
from urllib.request import urlopen
from io import StringIO
from contextlib import closing
from sklearn import preprocessing
from scipy.sparse.linalg import norm
from scipy.sparse import csr_matrix
from scipy.spatial.distance import pdist, squareform
import pandas as pd
import numpy as np
from app import app, tokenize, exclude, stopwords, MINDF
from db.db_get_data import get_dc_data

# import argparse
# import sys
# import operator
import math

# import gzip
# import random
import os
import time

time.clock = time.time
from datetime import datetime
from datetime import timedelta
from collections import defaultdict
import json

import pysolr
# from elasticsearch import Elasticsearch

# es = Elasticsearch()
import cluster_pymedAPI
import es_search

import logging
# Set logging level to suppress debug/info logs
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)

Reading stopwords...


DEBUG:git.cmd:Popen(['git', 'version'], cwd=/Users/aravind/Documents/LAIR Hub/CODE_v2/pattie_humanities, stdin=None, shell=False, universal_newlines=False)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/Users/aravind/Documents/LAIR Hub/CODE_v2/pattie_humanities, stdin=None, shell=False, universal_newlines=False)
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7


/Users/aravind/Documents/LAIR Hub/CODE_v2/pattie_humanities


In [2]:
from openai import OpenAI
from generate_summary import generate_summary

In [3]:
def cluster(raw_query_inp, num_cls_inp):
    print("cluster")
    error = None
    dataset = "PubMedAPI"

    field = "All fields"
    raw_query = raw_query_inp
    entity = "keywords"
    q = raw_query
    # print(q)
    num_cls = num_cls_inp

    if dataset == "PubMedAPI":
        print("PubMedAPI in routes.py")

        num_cls = int(num_cls)
        data, error = cluster_pymedAPI.retrieve(q)
        print(data, error)
        print(type(data), data.columns.tolist())
        if error == None:
            if entity == "genes":
                doc_term_mat, df, w2id, bibs = vl.read_df_genes(
                    session["gene_set"], data, dataset, stopwords
                )
                # print('-----------df------------')
                # print(df)
            elif entity == "authors":
                # doc_term_mat, df, w2id, bibs = vl.read_df_authors(
                #     data, dataset, stopwords)
                (
                    orig_doc_term_mat,
                    orig_df,
                    df,
                    dfr,
                    keywords,
                    cluster_centers,
                    cluster_desc,
                    coordinates,
                    id2members,
                    bibs,
                    org_ids,
                    author_names,
                    author_docs,
                ) = vl.process_authors_4(data, dataset, stopwords)
            else:
                doc_term_mat, df, w2id, bibs = vl.read_df(data, dataset, stopwords)
        else:
            # flash(error)
            print(error)
            
        if dataset != "Experts" and len(bibs) < 10:
            print("No clusters found. Broaden your search.")
            return
        
        if entity != "authors" and dataset != "Experts":
            print("Computing TFIDF and finding key terms...")
            # Remove terms whose df is lower than mindf
            if MINDF > 0:
                inf = []
                for w in df:
                    if df[w] <= MINDF:
                        inf.append(w)
                for w in inf:
                    del df[w]

            # Save org data
            orig_doc_term_mat = doc_term_mat
            orig_df = df

            # Compute tfidf and find key terms
            # print("Computing TFIDF and finding key terms...")
            if dataset == "NYTIMES" or dataset == "PLOS" or dataset == "DIABETES":
                doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=10)
            else:
                doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=30)

            # Sort and output results (discovered keywords)
            keywords = vl.output_keywords(len(doc_term_mat), dfr, df, p_docs=1.0)
            # print('Keywords...')
            # print(keywords)

            # Create new matrix with the keywords
            doc_term_mat, org_ids = vl.update(doc_term_mat, keywords)

            # Convert to sparse matrix
            doc_term_mat = vl.convert_sparse(doc_term_mat, keywords)

            # Clustering
            # print()
            # print("Clustering...")

            # n_components: number of dimensions for LSA
            # k: number of clusters
            # n_desc: number of keywords (desc) for each cluster

            if dataset == "PLOS":
                id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
                    doc_term_mat, keywords, org_ids, n_components=50, k=num_cls, n_desc=25
                )
            else:
                id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
                    doc_term_mat, keywords, org_ids, n_components=20, k=num_cls, n_desc=15
                )

            if error != None:  # needs to be changed
                print(error)
                flash(
                    "Sorry, the entity you've chosen does not generate meaningful clusters for current query. Please try other entities or queries. "
                )
                return redirect(request.referrer)
        
        # print("-----------------doc_term_mat-----------------")
        # print(doc_term_mat)
        # print("-----------------df-----------------")
        # print(df)
        # print("-----------------w2id-----------------")
        # print(w2id)
        # print("-----------------bibs-----------------")
        # print(bibs)
    
    # get cluster colors
    # center is (.5,.5)
    cc = np.array(cluster_centers) - 0.5
    hue = ((np.arctan2(cc[:, 1], cc[:, 0]) / np.pi + 1) * 180).astype(int).tolist()
    satr = (np.linalg.norm(cc, axis=1) / math.sqrt(2) * 2).tolist()
    val = [0.8] * num_cls

    # Create data to pass to result.html
    df_new = dict()
    dfr_new = dict()
    for w in keywords:
        df_new[w] = df[w]
        dfr_new[w] = dfr[w]

    # create adjacency matrix that will be used for network edges
    centroid_matrix = pd.DataFrame.from_records(cluster_centers)
    centroid_distances = pd.DataFrame(
        squareform(pdist(centroid_matrix, metric="euclidean")),
        columns=list(id2members.keys()),
        index=list(id2members.keys()),
    ).to_dict()

    id2freq = {x: len(id2members[x]) for x in id2members}

    # create network data structure
    # print('\nEUCLIDEAN DISTANCES\n----------')
    edges = []
    for i in id2members:
        for k, v in centroid_distances[i].items():
            # print(i, ' to ', k, ' = ', v)
            if i != k:
                if v > 0.75:
                    v = "Distant"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)
                elif v <= 0.75 and v > 0.50:
                    v = "Similar"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)
                elif v <= 0.50:
                    v = "Very Similar"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)

                # elif v >= 0.50:
                #     v = "Not Similar"
                # edge = {"clusterID":i,"source":cluster_centers[i],"target":cluster_centers[k],"distance":v}
                # edges.append(edge)
    sources = []
    id2meshTerms = {}
    if dataset == "PubMedAPI":
        from collections import Counter

        for cluster_id in sorted(id2members.keys()):
            concepts = cluster_desc[cluster_id]
            references = []
            for bib_id in id2members[cluster_id]:
                references.append(bibs[bib_id])
            for paper in references:
                if type(paper["author"]) != str and None in paper["author"]:
                    paper["author"] = [i for i in paper["author"] if i]
            bibliography = {"concepts": concepts, "references": references}
            meshTerms = []
            print("references")
            print(references)
            for ref in references:
                meshHeadingList = ref.get("meshHeadings")  # ref["meshHeadings"]
                for mesh in meshHeadingList:
                    meshTerms.append(mesh["descriptor"])
            id2meshTerms[cluster_id] = str(dict(Counter(meshTerms).most_common(20)))
            sources.append(bibliography)
        # print(sources)
    # session['doc_term_mat'] = doc_term_mat  # term-doc matrix
    session = {}
    session["docs_org"] = orig_doc_term_mat  # doc-term raw freq matrix
    session["df_org"] = orig_df
    session["num_cls"] = num_cls
    session["id2members_0"] = id2members
    session["cluster_desc_0"] = cluster_desc
    session["xy_0"] = cluster_centers
    # session["state"] = state
    session["hue_0"] = hue
    session["satr_0"] = satr
    session["org_ids_0"] = org_ids
    session["bibs_0"] = bibs
    session["dataset"] = dataset
    session["edges_0"] = edges
    session["sources"] = sources
    session["id2meshTerms"] = id2meshTerms

    session["entity"] = entity
    if entity == "authors":
        session["author_names"] = author_names
        session["author_docs"] = author_docs
        
    # Convert NumPy arrays to lists
    def convert_ndarray_to_list(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, list):
            return [convert_ndarray_to_list(item) for item in obj]
        elif isinstance(obj, dict):
            return {key: convert_ndarray_to_list(value) for key, value in obj.items()}
        else:
            return obj

    # Ensure orig_doc_term_mat is a list
    orig_doc_term_mat_list = convert_ndarray_to_list(orig_doc_term_mat)
    # Ensure orig_df is a list or dict (depending on its original type)
    orig_df_list = convert_ndarray_to_list(orig_df)
    # Ensure cluster_centers is a list
    cluster_centers_list = convert_ndarray_to_list(cluster_centers)
    # Ensure coordinates is a list
    coordinates_list = convert_ndarray_to_list(coordinates)


    sessionData = {
        "docs_org": convert_ndarray_to_list(orig_doc_term_mat),  # doc-term raw freq matrix
        "df_org": convert_ndarray_to_list(orig_df),
        "num_cls": num_cls,
        "id2members_0": convert_ndarray_to_list(id2members),
        "cluster_desc_0": convert_ndarray_to_list(cluster_desc),
        "xy_0": convert_ndarray_to_list(cluster_centers),
        # "state": state,
        "hue_0": convert_ndarray_to_list(hue),
        "satr_0": convert_ndarray_to_list(satr),
        "org_ids_0": convert_ndarray_to_list(org_ids),
        "dataset": dataset,
        "edges_0": convert_ndarray_to_list(edges),
        "sources": convert_ndarray_to_list(sources)
    }

    # session['df'] = df_new
    # session['dfr'] = dfr_new
    # session['keywords'] = keywords

    # print("bibs")
    # print(bibs)

    # output the data to terminal
    print('\nCENTROIDS\n----------')
    print(cluster_centers)
    print('\nCENTROID DISTANCES\n----------')
    print(centroid_distances)
    print('\nCLUSTER LABELS\n----------')
    print(cluster_desc)
    print('\nCLUSTER ID & PUBLICATIONS\n----------')
    print(id2members)
    print('\nCLUSTER ID & PUBLICATION COUNT\n----------')
    print(id2freq)
    print('\nNETWORK\n----------')
    print(edges)
    
    # Generate summaries for each cluster
    cluster_summary = {}
    cluster_assigned = {}
    for cluster_id in sorted(id2members.keys()):
        publications = id2members[cluster_id]
        pub_texts = "\n".join([f"@article{{{bib_id}, title={bibs[bib_id]['title']}}}" for bib_id in publications])
        user_content = f"Clustered Publications:\n\n{pub_texts}\n\nAssigned Cluster: '{cluster_desc[cluster_id]}'"
        # print(f"Cluster {cluster_id} \n\n User Content: {user_content}")
        summary = generate_summary(user_content)
        print(f"Cluster {cluster_id}, Assigned Cluster: '{cluster_desc[cluster_id]}'\n\nSummary: {summary}")
        cluster_summary[cluster_id] = summary
        cluster_assigned[cluster_id] = cluster_desc[cluster_id]



    if entity == "authors" or entity == "experts" or entity == "topics":
        # to tell the difference between keyword search and author search so that sessionStorage won't keep old data when switching between them
        q = q + "_" + entity

    print("exit cluster")
    return sessionData, cluster_summary, cluster_assigned, bibs, cluster_desc
    
ses_data, cluster_summary, cluster_assigned, bibs, cluster_desc = cluster("Fei Yu", 10)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443


cluster
PubMedAPI in routes.py
Retrieving data from PubMed


DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/esearch.fcgi?tool=PATTIE&email=nibras.rakib%40mail.utoronto.ca&db=pubmed&term=Fei+Yu&retmax=300&retmode=json HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?tool=PATTIE&email=nibras.rakib%40mail.utoronto.ca&db=pubmed&id=39068497&id=39065098&id=39048928&id=39047084&id=39045313&id=39042505&id=39039513&id=39035354&id=39032697&id=39029771&id=39024040&id=39018421&id=39015361&id=38992675&id=38990325&id=38984807&id=38981857&id=38978048&id=38977905&id=38975298&id=38969409&id=38953634&id=38950519&id=38949397&id=38942189&id=38941917&id=38925411&id=38914695&id=38905100&id=38901733&id=38897173&id=38890248&id=38889048&id=38878672&id=38878412&id=38875577&id=38871478&id=38860211&id=38858958&id=38858912&id=38858393&id=38852399&id=38849239&id=38839038&id=38828289

0      [Xuan Zhang, Haoxian Tang, Jingjing Chen, Jiny...
1      [Pi Chen, Zhen Li, Ning Cao, Rui-Xuan Wu, Zhao...
2      [Fei Yu, Mingming Zhang, Yueqin Yang, Yang Wan...
3      [Fei Yu, Xuan Zhao, Qian Wang, Po-Han Fang, Li...
4      [Ting Jiang, Fei Yu, Yuqi Zhou, Ruomei Li, Men...
                             ...                        
295    [Lingli Wu, Bin Ma, Fei Yu, Zhongming Ma, Qing...
296    [Huibin Long, Ruiqi Cao, Heyong Yin, Fei Yu, A...
297    [Anna R Kahkoska, Nikki L B Freeman, Emily P J...
298    [Guoqing Li, Su Liu, Yixiao Chen, Huihui Xu, T...
299    [Su Liu, Guoqing Li, Yuanchao Zhu, Chang Xu, Q...
Name: authors, Length: 300, dtype: object <class 'pandas.core.series.Series'>
Going to return df
                                               authors  \
0    [Xuan Zhang, Haoxian Tang, Jingjing Chen, Jiny...   
1    [Pi Chen, Zhen Li, Ning Cao, Rui-Xuan Wu, Zhao...   
2    [Fei Yu, Mingming Zhang, Yueqin Yang, Yang Wan...   
3    [Fei Yu, Xuan Zhao, Qian Wang, Po-Han F

/Users/aravind/anaconda3/envs/PATTIE_v2/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:1162: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


Extracted 227.0 coordinates
Extracted 16.0 coordinates
references
[{'pmid': '39042505', 'title': 'Electromagnetic radiation control for nonlinear dynamics of Hopfield neural networks.', 'author': ['Wei Yao', 'Jia Fang', 'Fei Yu', 'Li Xiong', 'Lihong Tang', 'Jin Zhang', 'Yichuang Sun'], 'affiliations': ['School of Computer and Communication Engineering, Changsha University of Science and Technology, Changsha, Hunan 410114, China.\nState Key Laboratory of Industrial Control Technology, Zhejiang University, Hangzhou 310058, China.\nKey Laboratory of Computing and Stochastic Mathematics (Ministry of Education), School of Mathematics and Statistics, Hunan Normal University, Changsha, Hunan 410081, China.', 'School of Computer and Communication Engineering, Changsha University of Science and Technology, Changsha, Hunan 410114, China.', 'School of Computer and Communication Engineering, Changsha University of Science and Technology, Changsha, Hunan 410114, China.', 'School of Physics and Elec

In [5]:
def cluster_postgres(raw_query_inp, num_cls_inp):
    print("cluster")
    error = None
    dataset = "postgres"

    field = "All fields"
    raw_query = raw_query_inp
    entity = "keywords"
    q = raw_query
    # print(q)
    num_cls = num_cls_inp
    
    if dataset == "postgres":
        print("postgres in routes.py")

        num_cls = int(num_cls)
        data, error = cluster_pymedAPI.retrieve(q)
        # print(data, error)
        print(type(data), data.columns.tolist())
        return data, error
        
data, error = cluster_postgres("Fei Yu", 10)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443


cluster
postgres in routes.py
Retrieving data from PubMed


DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/esearch.fcgi?tool=PATTIE&email=nibras.rakib%40mail.utoronto.ca&db=pubmed&term=Fei+Yu&retmax=300&retmode=json HTTP/11" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): eutils.ncbi.nlm.nih.gov:443
DEBUG:urllib3.connectionpool:https://eutils.ncbi.nlm.nih.gov:443 "GET /entrez/eutils/efetch.fcgi?tool=PATTIE&email=nibras.rakib%40mail.utoronto.ca&db=pubmed&id=39068497&id=39065098&id=39048928&id=39047084&id=39045313&id=39042505&id=39039513&id=39035354&id=39032697&id=39029771&id=39024040&id=39018421&id=39015361&id=38992675&id=38990325&id=38984807&id=38981857&id=38978048&id=38977905&id=38975298&id=38969409&id=38953634&id=38950519&id=38949397&id=38942189&id=38941917&id=38925411&id=38914695&id=38905100&id=38901733&id=38897173&id=38890248&id=38889048&id=38878672&id=38878412&id=38875577&id=38871478&id=38860211&id=38858958&id=38858912&id=38858393&id=38852399&id=38849239&id=38839038&id=38828289

0      [Xuan Zhang, Haoxian Tang, Jingjing Chen, Jiny...
1      [Pi Chen, Zhen Li, Ning Cao, Rui-Xuan Wu, Zhao...
2      [Fei Yu, Mingming Zhang, Yueqin Yang, Yang Wan...
3      [Fei Yu, Xuan Zhao, Qian Wang, Po-Han Fang, Li...
4      [Ting Jiang, Fei Yu, Yuqi Zhou, Ruomei Li, Men...
                             ...                        
295    [Lingli Wu, Bin Ma, Fei Yu, Zhongming Ma, Qing...
296    [Huibin Long, Ruiqi Cao, Heyong Yin, Fei Yu, A...
297    [Anna R Kahkoska, Nikki L B Freeman, Emily P J...
298    [Guoqing Li, Su Liu, Yixiao Chen, Huihui Xu, T...
299    [Su Liu, Guoqing Li, Yuanchao Zhu, Chang Xu, Q...
Name: authors, Length: 300, dtype: object <class 'pandas.core.series.Series'>
Going to return df
<class 'pandas.core.frame.DataFrame'> ['authors', 'affiliations', 'title', 'abstract', 'pmid', 'url', 'pubYear', 'pubdate', 'meshHeadings']


In [6]:
data

,authors,affiliations,title,abstract,pmid,url,pubYear,pubdate,meshHeadings
0,"[Xuan Zhang, Haoxian Tang, Jingjing Chen, Jiny...","[Department of Bone & Joint Surgery, Peking Un...",Association between different triglyceride-glu...,Insulin resistance (IR) and arthritis are stro...,39068497,https://www.ncbi.nlm.nih.gov/pubmed/?term=3906...,2024,2024-07-28,"[{'descriptor': 'Humans'}, {'descriptor': 'Fem..."
1,"[Pi Chen, Zhen Li, Ning Cao, Rui-Xuan Wu, Zhao...","[College of Forestry, Shanxi Agricultural Univ...",Comparison of Bacterial Communities in Five Ec...,Ectomycorrhizal fungi have huge potential valu...,39065098,https://www.ncbi.nlm.nih.gov/pubmed/?term=3906...,2024,2024-07-27,[]
2,"[Fei Yu, Mingming Zhang, Yueqin Yang, Yang Wan...","[College of Life Sciences, Henan Normal Univer...",Seed size and dispersal mode select mast seedi...,Reproduction by perennial plants varies from b...,39048928,https://www.ncbi.nlm.nih.gov/pubmed/?term=3904...,2024,2024-07-26,[]
3,"[Fei Yu, Xuan Zhao, Qian Wang, Po-Han Fang, Li...","[State Key Laboratory of Ophthalmology, Zhongs...",Engineered Mesenchymal Stromal Cell Exosomes-L...,Corneal alkali burns represent a prevalent oph...,39047084,https://www.ncbi.nlm.nih.gov/pubmed/?term=3904...,2024,2024-07-26,[]
4,"[Ting Jiang, Fei Yu, Yuqi Zhou, Ruomei Li, Men...","[Department of Orthodontics, Shanghai Ninth Pe...",Synergistic effect of ultrasound and reinforce...,The periosteum plays a vital role in repairing...,39045313,https://www.ncbi.nlm.nih.gov/pubmed/?term=3904...,2024,2024-07-24,[]
...,...,...,...,...,...,...,...,...,...
295,"[Lingli Wu, Bin Ma, Fei Yu, Zhongming Ma, Qing...","[Department of Dentistry, Key Laboratory of Or...",Salivary microbiome diversity in Chinese child...,This study aimed to explore oral microbiome di...,36538092,https://www.ncbi.nlm.nih.gov/pubmed/?term=3653...,2023,2022-12-21,"[{'descriptor': 'Child'}, {'descriptor': 'Chil..."
296,"[Huibin Long, Ruiqi Cao, Heyong Yin, Fei Yu, A...","[Department of Orthopedics, Beijing Friendship...","Associations between obesity, diabetes mellitu...",Data on common comorbidities targeting at diff...,36525243,https://www.ncbi.nlm.nih.gov/pubmed/?term=3652...,2023,2022-12-17,"[{'descriptor': 'Humans'}, {'descriptor': 'Ost..."
297,"[Anna R Kahkoska, Nikki L B Freeman, Emily P J...","[Department of Nutrition, University of North ...",Individualized interventions and precision hea...,Older adults are characterized by profound cli...,36524627,https://www.ncbi.nlm.nih.gov/pubmed/?term=3652...,2023,2022-12-17,"[{'descriptor': 'Humans'}, {'descriptor': 'Age..."
298,"[Guoqing Li, Su Liu, Yixiao Chen, Huihui Xu, T...","[Department of Bone & Joint Surgery, Peking Un...",Teriparatide ameliorates articular cartilage d...,Knee osteoarthritis (KOA) is a highly prevalen...,36514714,https://www.ncbi.nlm.nih.gov/pubmed/?term=3651...,2023,2022-12-15,[]


In [ ]:
def cluster_postgres(raw_query_inp, num_cls_inp):
    print("cluster")
    error = None
    dataset = "postgres"

    field = "All fields"
    raw_query = raw_query_inp
    entity = "keywords"
    q = raw_query
    # print(q)
    num_cls = num_cls_inp

    if dataset == "postgres":
        print("postgres in routes.py")

        num_cls = int(num_cls)
        data, error = cluster_pymedAPI.retrieve(q)
        print(data, error)
        print(type(data), data.columns.tolist())
        # if error == None:
        #     if entity == "genes":
        #         doc_term_mat, df, w2id, bibs = vl.read_df_genes(
        #             session["gene_set"], data, dataset, stopwords
        #         )
        #         # print('-----------df------------')
        #         # print(df)
        #     elif entity == "authors":
        #         # doc_term_mat, df, w2id, bibs = vl.read_df_authors(
        #         #     data, dataset, stopwords)
        #         (
        #             orig_doc_term_mat,
        #             orig_df,
        #             df,
        #             dfr,
        #             keywords,
        #             cluster_centers,
        #             cluster_desc,
        #             coordinates,
        #             id2members,
        #             bibs,
        #             org_ids,
        #             author_names,
        #             author_docs,
        #         ) = vl.process_authors_4(data, dataset, stopwords)
        #     else:
        #         doc_term_mat, df, w2id, bibs = vl.read_df(data, dataset, stopwords)
        # else:
        #     # flash(error)
        #     print(error)
            
        # if dataset != "Experts" and len(bibs) < 10:
        #     print("No clusters found. Broaden your search.")
        #     return
        
        # if entity != "authors" and dataset != "Experts":
        #     print("Computing TFIDF and finding key terms...")
        #     # Remove terms whose df is lower than mindf
        #     if MINDF > 0:
        #         inf = []
        #         for w in df:
        #             if df[w] <= MINDF:
        #                 inf.append(w)
        #         for w in inf:
        #             del df[w]

        #     # Save org data
        #     orig_doc_term_mat = doc_term_mat
        #     orig_df = df

        #     # Compute tfidf and find key terms
        #     # print("Computing TFIDF and finding key terms...")
        #     if dataset == "NYTIMES" or dataset == "PLOS" or dataset == "DIABETES":
        #         doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=10)
        #     else:
        #         doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=30)

        #     # Sort and output results (discovered keywords)
        #     keywords = vl.output_keywords(len(doc_term_mat), dfr, df, p_docs=1.0)
        #     # print('Keywords...')
        #     # print(keywords)

        #     # Create new matrix with the keywords
        #     doc_term_mat, org_ids = vl.update(doc_term_mat, keywords)

        #     # Convert to sparse matrix
        #     doc_term_mat = vl.convert_sparse(doc_term_mat, keywords)

        #     # Clustering
        #     # print()
        #     # print("Clustering...")

        #     # n_components: number of dimensions for LSA
        #     # k: number of clusters
        #     # n_desc: number of keywords (desc) for each cluster

        #     if dataset == "PLOS":
        #         id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
        #             doc_term_mat, keywords, org_ids, n_components=50, k=num_cls, n_desc=25
        #         )
        #     else:
        #         id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
        #             doc_term_mat, keywords, org_ids, n_components=20, k=num_cls, n_desc=15
        #         )

        #     if error != None:  # needs to be changed
        #         print(error)
        #         flash(
        #             "Sorry, the entity you've chosen does not generate meaningful clusters for current query. Please try other entities or queries. "
        #         )
        #         return redirect(request.referrer)
        
        # # print("-----------------doc_term_mat-----------------")
        # # print(doc_term_mat)
        # # print("-----------------df-----------------")
        # # print(df)
        # # print("-----------------w2id-----------------")
        # # print(w2id)
        # # print("-----------------bibs-----------------")
        # # print(bibs)
        
    # get cluster colors
    # center is (.5,.5)
    cc = np.array(cluster_centers) - 0.5
    hue = ((np.arctan2(cc[:, 1], cc[:, 0]) / np.pi + 1) * 180).astype(int).tolist()
    satr = (np.linalg.norm(cc, axis=1) / math.sqrt(2) * 2).tolist()
    val = [0.8] * num_cls

    # Create data to pass to result.html
    df_new = dict()
    dfr_new = dict()
    for w in keywords:
        df_new[w] = df[w]
        dfr_new[w] = dfr[w]

    # create adjacency matrix that will be used for network edges
    centroid_matrix = pd.DataFrame.from_records(cluster_centers)
    centroid_distances = pd.DataFrame(
        squareform(pdist(centroid_matrix, metric="euclidean")),
        columns=list(id2members.keys()),
        index=list(id2members.keys()),
    ).to_dict()

    id2freq = {x: len(id2members[x]) for x in id2members}

    # create network data structure
    # print('\nEUCLIDEAN DISTANCES\n----------')
    edges = []
    for i in id2members:
        for k, v in centroid_distances[i].items():
            # print(i, ' to ', k, ' = ', v)
            if i != k:
                if v > 0.75:
                    v = "Distant"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)
                elif v <= 0.75 and v > 0.50:
                    v = "Similar"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)
                elif v <= 0.50:
                    v = "Very Similar"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)

                # elif v >= 0.50:
                #     v = "Not Similar"
                # edge = {"clusterID":i,"source":cluster_centers[i],"target":cluster_centers[k],"distance":v}
                # edges.append(edge)
    sources = []
    id2meshTerms = {}
    if dataset == "PubMedAPI":
        from collections import Counter

        for cluster_id in sorted(id2members.keys()):
            concepts = cluster_desc[cluster_id]
            references = []
            for bib_id in id2members[cluster_id]:
                references.append(bibs[bib_id])
            for paper in references:
                if type(paper["author"]) != str and None in paper["author"]:
                    paper["author"] = [i for i in paper["author"] if i]
            bibliography = {"concepts": concepts, "references": references}
            meshTerms = []
            print("references")
            print(references)
            for ref in references:
                meshHeadingList = ref.get("meshHeadings")  # ref["meshHeadings"]
                for mesh in meshHeadingList:
                    meshTerms.append(mesh["descriptor"])
            id2meshTerms[cluster_id] = str(dict(Counter(meshTerms).most_common(20)))
            sources.append(bibliography)
        # print(sources)
    # session['doc_term_mat'] = doc_term_mat  # term-doc matrix
    session = {}
    session["docs_org"] = orig_doc_term_mat  # doc-term raw freq matrix
    session["df_org"] = orig_df
    session["num_cls"] = num_cls
    session["id2members_0"] = id2members
    session["cluster_desc_0"] = cluster_desc
    session["xy_0"] = cluster_centers
    # session["state"] = state
    session["hue_0"] = hue
    session["satr_0"] = satr
    session["org_ids_0"] = org_ids
    session["bibs_0"] = bibs
    session["dataset"] = dataset
    session["edges_0"] = edges
    session["sources"] = sources
    session["id2meshTerms"] = id2meshTerms

    session["entity"] = entity
    if entity == "authors":
        session["author_names"] = author_names
        session["author_docs"] = author_docs
        
    # Convert NumPy arrays to lists
    def convert_ndarray_to_list(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, list):
            return [convert_ndarray_to_list(item) for item in obj]
        elif isinstance(obj, dict):
            return {key: convert_ndarray_to_list(value) for key, value in obj.items()}
        else:
            return obj

    # Ensure orig_doc_term_mat is a list
    orig_doc_term_mat_list = convert_ndarray_to_list(orig_doc_term_mat)
    # Ensure orig_df is a list or dict (depending on its original type)
    orig_df_list = convert_ndarray_to_list(orig_df)
    # Ensure cluster_centers is a list
    cluster_centers_list = convert_ndarray_to_list(cluster_centers)
    # Ensure coordinates is a list
    coordinates_list = convert_ndarray_to_list(coordinates)


    sessionData = {
        "docs_org": convert_ndarray_to_list(orig_doc_term_mat),  # doc-term raw freq matrix
        "df_org": convert_ndarray_to_list(orig_df),
        "num_cls": num_cls,
        "id2members_0": convert_ndarray_to_list(id2members),
        "cluster_desc_0": convert_ndarray_to_list(cluster_desc),
        "xy_0": convert_ndarray_to_list(cluster_centers),
        # "state": state,
        "hue_0": convert_ndarray_to_list(hue),
        "satr_0": convert_ndarray_to_list(satr),
        "org_ids_0": convert_ndarray_to_list(org_ids),
        "dataset": dataset,
        "edges_0": convert_ndarray_to_list(edges),
        "sources": convert_ndarray_to_list(sources)
    }

    # session['df'] = df_new
    # session['dfr'] = dfr_new
    # session['keywords'] = keywords

    # print("bibs")
    # print(bibs)

    # output the data to terminal
    print('\nCENTROIDS\n----------')
    print(cluster_centers)
    print('\nCENTROID DISTANCES\n----------')
    print(centroid_distances)
    print('\nCLUSTER LABELS\n----------')
    print(cluster_desc)
    print('\nCLUSTER ID & PUBLICATIONS\n----------')
    print(id2members)
    print('\nCLUSTER ID & PUBLICATION COUNT\n----------')
    print(id2freq)
    print('\nNETWORK\n----------')
    print(edges)
    
    # Generate summaries for each cluster
    cluster_summary = {}
    cluster_assigned = {}
    for cluster_id in sorted(id2members.keys()):
        publications = id2members[cluster_id]
        pub_texts = "\n".join([f"@article{{{bib_id}, title={bibs[bib_id]['title']}}}" for bib_id in publications])
        user_content = f"Clustered Publications:\n\n{pub_texts}\n\nAssigned Cluster: '{cluster_desc[cluster_id]}'"
        # print(f"Cluster {cluster_id} \n\n User Content: {user_content}")
        summary = generate_summary(user_content)
        print(f"Cluster {cluster_id}, Assigned Cluster: '{cluster_desc[cluster_id]}'\n\nSummary: {summary}")
        cluster_summary[cluster_id] = summary
        cluster_assigned[cluster_id] = cluster_desc[cluster_id]



    if entity == "authors" or entity == "experts" or entity == "topics":
        # to tell the difference between keyword search and author search so that sessionStorage won't keep old data when switching between them
        q = q + "_" + entity

    print("exit cluster")
    return sessionData, cluster_summary, cluster_assigned, bibs, cluster_desc
    
ses_data, cluster_summary, cluster_assigned, bibs, cluster_desc = cluster("Fei Yu", 10)

In [56]:
len(cluster_desc)
cluster_desc

[['patients',
  'median',
  'group',
  'study',
  'groups',
  'coronary',
  'significantly',
  'analysis',
  'regression',
  'follow-up',
  'rate',
  'adverse',
  'higher',
  'risk'],
 ['cell',
  'expression',
  'role',
  'development',
  'cells',
  'mechanisms',
  'protein',
  'gene',
  'therapeutic',
  'potential',
  'pathway',
  'signaling',
  'regulatory',
  'proliferation',
  'treatment'],
 ['training',
  'model',
  'dataset',
  'learning',
  'curve',
  'features',
  'radiomics',
  'degs',
  'predictive',
  'machine',
  'models',
  'genes'],
 ['biodegradation',
  'p450',
  'pops',
  'fungi',
  'cytochrome',
  'environmental',
  'bacteria',
  'communities',
  'prospects',
  'engineering',
  'environments',
  'microbial',
  'mangrove',
  'concern',
  'enriched'],
 ['capacitive',
  'deionization',
  'storage',
  'prepared',
  'applications',
  'materials',
  'electrodes',
  'electrode',
  'fundamental',
  'application',
  'metal-organic',
  'conductivity',
  'preparation'],
 ['fiber'

In [59]:
# Convert the dictionary cluster_summary to a 2D lsit
cluster_summary_list = [[value] for key, value in cluster_summary.items()]
cluster_summary_list


[['Clinical studies analyze patient groups with varying coronary risk factors, predicting adverse outcomes.'],
 ['Cellular mechanisms and gene regulation for therapeutic potential in cell signaling pathways.'],
 ['Predictive models utilizing radiomics and gene features for disease prognosis.'],
 ['Microbial communities in diverse environments for biodegradation and environmental prospects.'],
 ['Innovative metal-organic materials enhance capacitive deionization for water treatment.'],
 ['Advanced optical spectroscopy using anti-resonant hollow-core fibers for precise wavelength modulation.'],
 ['Metagenomic sequencing aids in pneumonia diagnosis and identifies pathogens accurately.'],
 ['Associations of glaucoma with race, ethnicity, and population in California Medicare.'],
 ['Research in China revealed significant treatment effects and increased risks based on analysis.'],
 ['Surgical interventions for rare hand complications with limb salvage techniques.']]

In [60]:
cluster_summary

{0: 'Clinical studies analyze patient groups with varying coronary risk factors, predicting adverse outcomes.',
 1: 'Cellular mechanisms and gene regulation for therapeutic potential in cell signaling pathways.',
 2: 'Predictive models utilizing radiomics and gene features for disease prognosis.',
 3: 'Microbial communities in diverse environments for biodegradation and environmental prospects.',
 4: 'Innovative metal-organic materials enhance capacitive deionization for water treatment.',
 5: 'Advanced optical spectroscopy using anti-resonant hollow-core fibers for precise wavelength modulation.',
 6: 'Metagenomic sequencing aids in pneumonia diagnosis and identifies pathogens accurately.',
 7: 'Associations of glaucoma with race, ethnicity, and population in California Medicare.',
 8: 'Research in China revealed significant treatment effects and increased risks based on analysis.',
 9: 'Surgical interventions for rare hand complications with limb salvage techniques.'}

In [48]:
cluster_assigned

{0: ['patients',
  'median',
  'group',
  'study',
  'groups',
  'coronary',
  'significantly',
  'analysis',
  'regression',
  'follow-up',
  'rate',
  'adverse',
  'higher',
  'risk'],
 1: ['cell',
  'expression',
  'role',
  'development',
  'cells',
  'mechanisms',
  'protein',
  'gene',
  'therapeutic',
  'potential',
  'pathway',
  'signaling',
  'regulatory',
  'proliferation',
  'treatment'],
 2: ['training',
  'model',
  'dataset',
  'learning',
  'curve',
  'features',
  'radiomics',
  'degs',
  'predictive',
  'machine',
  'models',
  'genes'],
 3: ['biodegradation',
  'p450',
  'pops',
  'fungi',
  'cytochrome',
  'environmental',
  'bacteria',
  'communities',
  'prospects',
  'engineering',
  'environments',
  'microbial',
  'mangrove',
  'concern',
  'enriched'],
 4: ['capacitive',
  'deionization',
  'storage',
  'prepared',
  'applications',
  'materials',
  'electrodes',
  'electrode',
  'fundamental',
  'application',
  'metal-organic',
  'conductivity',
  'preparati

In [3]:
import psycopg2
import pandas as pd
from psycopg2.extras import RealDictCursor

def execute_query(query, hostname, port, database, username, password):
    error = None
    # Database connection parameters
    # hostname = 'localhost'
    # port = 5433
    # database = 'PATTIE'
    # username = 'postgres'
    # password = 'Arvi1308'  # replace with your actual password
    if query is None:
        error = "Query is empty"
        return [], error
    try:
        # Establishing the connection
        conn = psycopg2.connect(
            host=hostname,
            port=port,
            dbname=database,
            user=username,
            password=password
        )

        # Create a cursor object using RealDictCursor to get results as dictionaries
        # cur = conn.cursor(cursor_factory=RealDictCursor)
        cur = conn.cursor()

        # Execute the query
        cur.execute(query)

        # Fetch all the results
        results = cur.fetchall()

        # Close the cursor and the connection
        cur.close()
        conn.close()

        # Return the results
        return results, error

    except Exception as e:
        error = f"Unexpected error: {e}"
        print(f"An error occurred: {e}")
        if conn:
            conn.close()
        return [], error


def retrieve_from_postgresql(author_name):
    # Database connection parameters
    # Host: 34.133.177.246
    # Port: 5432
    # User: aravind
    # Password: C&99Fk6xHxypA2R$C4XQ
    # Database name: dcdi
    # hostname = 'localhost'
    # port = 5433
    # database = 'PATTIE'
    # username = 'postgres'
    # password = 'Arvi1308'  # replace with your actual password
    hostname = '34.133.177.246'
    port = 5432
    database = 'dcdi'
    username = 'aravind'
    password = 'C&99Fk6xHxypA2R$C4XQ'

    query = f"""
        SELECT title, abstract, authors, url
        FROM papers
        WHERE authors::jsonb @> '[\"{author_name}\"]';
        """
    rows, error = execute_query(query, hostname, port, database, username, password)
    print("ROWS:")
    print(rows)
    if rows:
        # Process each row to ensure consistent structure
        processed_rows = []
        for row in rows:
            # Debugging: Print the structure of each row
            print("Row structure:", type(row), row)
            
            # Each row is a tuple, ensure it has all columns, replace None with empty string
            processed_row = tuple(item if item is not None else '' for item in row)
            processed_rows.append(processed_row)
        
        if processed_rows:
            # Define columns as per the table structure
            columns = [
                'title', 'abstract', 'authors', 'url'
            ]
            # Create a DataFrame from the processed results
            df = pd.DataFrame(processed_rows, columns=columns)
    else:
        df = pd.DataFrame()
    
    return df, error

# Example usage
author_name = 'Amanda Visconti'
df, error = retrieve_from_postgresql(author_name)
if error:
    print(error)
else:
    print(df)

# # Example usage
# search_word = 'digital'
# df, error = retrieve_from_postgresql(search_word)
# if error:
#     print(error)
# else:
#     print(df)



ROWS:
[("'Songs of Innocence and of Experience:' Amateur Users and Digital Texts", "Digital texts promise to allow learning beyond that possible with traditional resources. Purposebuilt digital texts are crafted for specific research purposes, with developer-users and devoted academics comprising their primary, 'scholar' audience. A secondary, 'amateur' audience of learners with less digital text experience also relies on theses purpose-built resources. Does the promise of new learning from digital texts extend beyond scholars to amateurs, or does the design of purpose-built digital texts, by focusing on more experienced users with direct lines of communication to digital text developers, prevent this extension of benefits? This study gauged one subgroup of amateur users' perceptions of the value of digital texts in terms of answering self-generated research queries. The participants, graduate students from the University of Michigan's information master's program, worked with a digita

In [ ]:
# def cluster_postgres(raw_query_inp, num_cls_inp):
#     print("cluster")
#     error = None
#     dataset = "PostgreSQL"

#     field = "All fields"
#     raw_query = raw_query_inp
#     entity = "keywords"
#     q = raw_query
#     num_cls = num_cls_inp

#     if dataset == "PostgreSQL":
#         print("PostgreSQL in routes.py")

#         num_cls = int(num_cls)
#         data, error = retrieve_from_postgresql(q)
#         print(data, error)
#         print(type(data), data.columns.tolist())
#         if error == None:
#             if entity == "genes":
#                 doc_term_mat, df, w2id, bibs = vl.read_df_genes(
#                     session["gene_set"], data, dataset, stopwords
#                 )
#             elif entity == "authors":
#                 (
#                     orig_doc_term_mat,
#                     orig_df,
#                     df,
#                     dfr,
#                     keywords,
#                     cluster_centers,
#                     cluster_desc,
#                     coordinates,
#                     id2members,
#                     bibs,
#                     org_ids,
#                     author_names,
#                     author_docs,
#                 ) = vl.process_authors_4(data, dataset, stopwords)
#             else:
#                 doc_term_mat, df, w2id, bibs = vl.read_df(data, dataset, stopwords)
#         else:
#             print(error)
#             return

#         if dataset != "Experts" and len(bibs) < 10:
#             print("No clusters found. Broaden your search.")
#             return

#         if entity != "authors" and dataset != "Experts":
#             print("Computing TFIDF and finding key terms...")
#             if MINDF > 0:
#                 inf = []
#                 for w in df:
#                     if df[w] <= MINDF:
#                         inf.append(w)
#                 for w in inf:
#                     del df[w]

#             orig_doc_term_mat = doc_term_mat
#             orig_df = df

#             if dataset == "NYTIMES" or dataset == "PLOS" or dataset == "DIABETES":
#                 doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=10)
#             else:
#                 doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=30)

#             keywords = vl.output_keywords(len(doc_term_mat), dfr, df, p_docs=1.0)
#             doc_term_mat, org_ids = vl.update(doc_term_mat, keywords)
#             doc_term_mat = vl.convert_sparse(doc_term_mat, keywords)

#             if dataset == "PLOS":
#                 id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
#                     doc_term_mat, keywords, org_ids, n_components=50, k=num_cls, n_desc=25
#                 )
#             else:
#                 id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
#                     doc_term_mat, keywords, org_ids, n_components=20, k=num_cls, n_desc=15
#                 )

#             if error != None:
#                 print(error)
#                 return redirect(request.referrer)

#     cc = np.array(cluster_centers) - 0.5
#     hue = ((np.arctan2(cc[:, 1], cc[:, 0]) / np.pi + 1) * 180).astype(int).tolist()
#     satr = (np.linalg.norm(cc, axis=1) / math.sqrt(2) * 2).tolist()
#     val = [0.8] * num_cls

#     df_new = dict()
#     dfr_new = dict()
#     for w in keywords:
#         df_new[w] = df[w]
#         dfr_new[w] = dfr[w]

#     centroid_matrix = pd.DataFrame.from_records(cluster_centers)
#     centroid_distances = pd.DataFrame(
#         squareform(pdist(centroid_matrix, metric="euclidean")),
#         columns=list(id2members.keys()),
#         index=list(id2members.keys()),
#     ).to_dict()

#     id2freq = {x: len(id2members[x]) for x in id2members}

#     edges = []
#     for i in id2members:
#         for k, v in centroid_distances[i].items():
#             if i != k:
#                 if v > 0.75:
#                     v = "Distant"
#                     edge = {
#                         "clusterID": i,
#                         "source": cluster_centers[i],
#                         "target": cluster_centers[k],
#                         "distance": v,
#                     }
#                     edges.append(edge)
#                 elif v <= 0.75 and v > 0.50:
#                     v = "Similar"
#                     edge = {
#                         "clusterID": i,
#                         "source": cluster_centers[i],
#                         "target": cluster_centers[k],
#                         "distance": v,
#                     }
#                     edges.append(edge)
#                 elif v <= 0.50:
#                     v = "Very Similar"
#                     edge = {
#                         "clusterID": i,
#                         "source": cluster_centers[i],
#                         "target": cluster_centers[k],
#                         "distance": v,
#                     }
#                     edges.append(edge)
#     sources = []
#     id2meshTerms = {}
#     if dataset == "PostgreSQL":
#         from collections import Counter

#         for cluster_id in sorted(id2members.keys()):
#             concepts = cluster_desc[cluster_id]
#             references = []
#             print(f'--------{cluster_id}---------')
#             for bib_id in id2members[cluster_id]:
#                 references.append(bibs[bib_id])
#             print(references)
#             print('-----------------')
#             for paper in references:
#                 if type(paper["author"]) != str and None in paper["author"]:
#                     paper["author"] = [i for i in paper["author"] if i]
#                     print(paper["author"])
#             # Add 'meshHeadings': [] to each reference in references
#             for ref in references:
#                 ref['meshHeadings'] = []
#             bibliography = {"concepts": concepts, "references": references}
#             meshTerms = []
#             for ref in references:
#                 meshHeadingList = ref.get("meshHeadings")  # ref["meshHeadings"]
#                 for mesh in meshHeadingList:
#                     meshTerms.append(mesh["descriptor"])
#             id2meshTerms[cluster_id] = str(dict(Counter(meshTerms).most_common(20)))
#             sources.append(bibliography)

#     session = {}
#     session["docs_org"] = orig_doc_term_mat  # doc-term raw freq matrix
#     session["df_org"] = orig_df
#     session["num_cls"] = num_cls
#     session["id2members_0"] = id2members
#     session["cluster_desc_0"] = cluster_desc
#     session["xy_0"] = cluster_centers
#     session["hue_0"] = hue
#     session["satr_0"] = satr
#     session["org_ids_0"] = org_ids
#     session["bibs_0"] = bibs
#     session["dataset"] = dataset
#     session["edges_0"] = edges
#     session["sources"] = sources
#     session["id2meshTerms"] = id2meshTerms

#     session["entity"] = entity
#     if entity == "authors":
#         session["author_names"] = author_names
#         session["author_docs"] = author_docs

#     def convert_ndarray_to_list(obj):
#         if isinstance(obj, np.ndarray):
#             return obj.tolist()
#         elif isinstance(obj, list):
#             return [convert_ndarray_to_list(item) for item in obj]
#         elif isinstance(obj, dict):
#             return {key: convert_ndarray_to_list(value) for key, value in obj.items()}
#         else:
#             return obj

#     orig_doc_term_mat_list = convert_ndarray_to_list(orig_doc_term_mat)
#     orig_df_list = convert_ndarray_to_list(orig_df)
#     cluster_centers_list = convert_ndarray_to_list(cluster_centers)
#     coordinates_list = convert_ndarray_to_list(coordinates)

#     sessionData = {
#         "docs_org": convert_ndarray_to_list(orig_doc_term_mat),
#         "df_org": convert_ndarray_to_list(orig_df),
#         "num_cls": num_cls,
#         "id2members_0": convert_ndarray_to_list(id2members),
#         "cluster_desc_0": convert_ndarray_to_list(cluster_desc),
#         "xy_0": convert_ndarray_to_list(cluster_centers),
#         "hue_0": convert_ndarray_to_list(hue),
#         "satr_0": convert_ndarray_to_list(satr),
#         "org_ids_0": convert_ndarray_to_list(org_ids),
#         "dataset": dataset,
#         "edges_0": convert_ndarray_to_list(edges),
#         "sources": convert_ndarray_to_list(sources)
#     }

#     print('\nCENTROIDS\n----------')
#     print(cluster_centers)
#     print('\nCENTROID DISTANCES\n----------')
#     print(centroid_distances)
#     print('\nCLUSTER LABELS\n----------')
#     print(cluster_desc)
#     print('\nCLUSTER ID & PUBLICATIONS\n----------')
#     print(id2members)
#     print('\nCLUSTER ID & PUBLICATION COUNT\n----------')
#     print(id2freq)
#     print('\nNETWORK\n----------')
#     print(edges)

#     cluster_summary = {}
#     cluster_assigned = {}
#     for cluster_id in sorted(id2members.keys()):
#         publications = id2members[cluster_id]
#         pub_texts = "\n".join([f"@article{{{bib_id}, title={bibs[bib_id]['title']}}}" for bib_id in publications])
#         user_content = f"Clustered Publications:\n\n{pub_texts}\n\nAssigned Cluster: '{cluster_desc[cluster_id]}'"
#         summary = generate_summary(user_content)
#         print(f"Cluster {cluster_id}, Assigned Cluster: '{cluster_desc[cluster_id]}'\n\nSummary: {summary}")
#         cluster_summary[cluster_id] = summary
#         cluster_assigned[cluster_id] = cluster_desc[cluster_id]

#     if entity == "authors" or entity == "experts" or entity == "topics":
#         q = q + "_" + entity

#     print("exit cluster")
#     return sessionData, cluster_summary, cluster_assigned, bibs, cluster_desc



# ses_data, cluster_summary, cluster_assigned, bibs, cluster_desc = cluster_postgres("M. Kirschenbaum", 10)


In [4]:
def cluster_postgres(raw_query_inp, num_cls_inp):
    print("cluster")
    error = None
    dataset = "PostgreSQL"

    field = "All fields"
    raw_query = raw_query_inp
    entity = "keywords"
    q = raw_query
    num_cls = num_cls_inp
    state = 0
    resonse_time = "0"
    send_time = time.clock()
    
    if dataset == "PostgreSQL":
        print("PostgreSQL in routes.py")

        num_cls = int(num_cls)
        data, error = retrieve_from_postgresql(q)
        print(data, error)
        print(type(data), data.columns.tolist())
        if error == None:
            if entity == "genes":
                doc_term_mat, df, w2id, bibs = vl.read_df_genes(
                    session["gene_set"], data, dataset, stopwords
                )
            elif entity == "authors":
                (
                    orig_doc_term_mat,
                    orig_df,
                    df,
                    dfr,
                    keywords,
                    cluster_centers,
                    cluster_desc,
                    coordinates,
                    id2members,
                    bibs,
                    org_ids,
                    author_names,
                    author_docs,
                ) = vl.process_authors_4(data, dataset, stopwords)
            else:
                doc_term_mat, df, w2id, bibs = vl.read_df(data, dataset, stopwords)
        else:
            print(error)
            return

        if dataset != "Experts" and len(bibs) < 10:
            print("No clusters found. Broaden your search.")
            return

        if entity != "authors" and dataset != "Experts":
            print("Computing TFIDF and finding key terms...")
            if MINDF > 0:
                inf = []
                for w in df:
                    if df[w] <= MINDF:
                        inf.append(w)
                for w in inf:
                    del df[w]

            orig_doc_term_mat = doc_term_mat
            orig_df = df

            if dataset == "NYTIMES" or dataset == "PLOS" or dataset == "DIABETES":
                doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=10)
            else:
                doc_term_mat, dfr = vl.compute_tfidf(doc_term_mat, df, rank=30)

            keywords = vl.output_keywords(len(doc_term_mat), dfr, df, p_docs=1.0)
            doc_term_mat, org_ids = vl.update(doc_term_mat, keywords)
            doc_term_mat = vl.convert_sparse(doc_term_mat, keywords)

            if dataset == "PLOS":
                id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
                    doc_term_mat, keywords, org_ids, n_components=50, k=num_cls, n_desc=25
                )
            else:
                id2members, cluster_centers, cluster_desc, coordinates, error = vl.kmeans(
                    doc_term_mat, keywords, org_ids, n_components=20, k=num_cls, n_desc=15
                )

            if error != None:
                print(error)
                return redirect(request.referrer)

    cc = np.array(cluster_centers) - 0.5
    hue = ((np.arctan2(cc[:, 1], cc[:, 0]) / np.pi + 1) * 180).astype(int).tolist()
    satr = (np.linalg.norm(cc, axis=1) / math.sqrt(2) * 2).tolist()
    val = [0.8] * num_cls

    df_new = dict()
    dfr_new = dict()
    for w in keywords:
        df_new[w] = df[w]
        dfr_new[w] = dfr[w]

    centroid_matrix = pd.DataFrame.from_records(cluster_centers)
    centroid_distances = pd.DataFrame(
        squareform(pdist(centroid_matrix, metric="euclidean")),
        columns=list(id2members.keys()),
        index=list(id2members.keys()),
    ).to_dict()

    id2freq = {x: len(id2members[x]) for x in id2members}

    edges = []
    for i in id2members:
        for k, v in centroid_distances[i].items():
            if i != k:
                if v > 0.75:
                    v = "Distant"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)
                elif v <= 0.75 and v > 0.50:
                    v = "Similar"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)
                elif v <= 0.50:
                    v = "Very Similar"
                    edge = {
                        "clusterID": i,
                        "source": cluster_centers[i],
                        "target": cluster_centers[k],
                        "distance": v,
                    }
                    edges.append(edge)
    sources = []
    id2meshTerms = {}
    if dataset == "PostgreSQL":
        from collections import Counter

        for cluster_id in sorted(id2members.keys()):
            concepts = cluster_desc[cluster_id]
            references = []
            print(f'--------{cluster_id}---------')
            for bib_id in id2members[cluster_id]:
                references.append(bibs[bib_id])
            print(references)
            print('-----------------')
            for paper in references:
                if type(paper["author"]) != str and None in paper["author"]:
                    paper["author"] = [i for i in paper["author"] if i]
                    print(paper["author"])
            # Add 'meshHeadings': [] to each reference in references
            for ref in references:
                ref['meshHeadings'] = []
            bibliography = {"concepts": concepts, "references": references}
            meshTerms = []
            for ref in references:
                meshHeadingList = ref.get("meshHeadings")  # ref["meshHeadings"]
                for mesh in meshHeadingList:
                    meshTerms.append(mesh["descriptor"])
            id2meshTerms[cluster_id] = str(dict(Counter(meshTerms).most_common(20)))
            sources.append(bibliography)

    session = {}
    session["docs_org"] = orig_doc_term_mat  # doc-term raw freq matrix
    session["df_org"] = orig_df
    session["num_cls"] = num_cls
    session["id2members_0"] = id2members
    session["cluster_desc_0"] = cluster_desc
    session["xy_0"] = cluster_centers
    session["hue_0"] = hue
    session["satr_0"] = satr
    session["org_ids_0"] = org_ids
    session["bibs_0"] = bibs
    session["dataset"] = dataset
    session["edges_0"] = edges
    session["sources"] = sources
    session["id2meshTerms"] = id2meshTerms

    session["entity"] = entity
    if entity == "authors":
        session["author_names"] = author_names
        session["author_docs"] = author_docs

    def convert_ndarray_to_list(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, list):
            return [convert_ndarray_to_list(item) for item in obj]
        elif isinstance(obj, dict):
            return {key: convert_ndarray_to_list(value) for key, value in obj.items()}
        else:
            return obj

    orig_doc_term_mat_list = convert_ndarray_to_list(orig_doc_term_mat)
    orig_df_list = convert_ndarray_to_list(orig_df)
    cluster_centers_list = convert_ndarray_to_list(cluster_centers)
    coordinates_list = convert_ndarray_to_list(coordinates)
    
    # Generate summaries for each cluster
    cluster_summary = {}
    cluster_assigned = {}
    cluster_title = {}
    for cluster_id in sorted(id2members.keys()):
        publications = id2members[cluster_id]
        pub_texts = "\n".join([f"@article{{{bib_id}, title={bibs[bib_id]['title']}}}" for bib_id in publications])
        # user_content = f"Clustered Publications:\n\n{pub_texts}\n\nAssigned Cluster: '{cluster_desc[cluster_id]}'"
        user_content = f"Clustered Publications:\n\n{pub_texts}'"
        # print(f"Cluster {cluster_id} \n\n User Content: {user_content}")
        print(f"Cluster {cluster_id} \n\n User Content: {user_content}\n\n")
        summary = generate_summary(user_content)
        # Split the string into parts based on the numbering
        print("Preprocessed Summary: ", summary, "\n\n")
        parts = summary.split("2. ")

        summary_title = parts[0].replace("1. ", "").strip()
        summary_val = parts[1].strip()
        print(f"Cluster {cluster_id}, Assigned Cluster: '{cluster_desc[cluster_id]}'\n\nSummary: {summary_val}\n\nTitle: {summary_title}\n\n")
        if 'N/A' in summary_val or 'N/A' in summary_title:
            summary_val = "No summary available due to insufficient paper content"
            summary_title = "No title available"
        cluster_summary[cluster_id] = summary_val
        cluster_assigned[cluster_id] = cluster_desc[cluster_id]
        # If summary title is a sentence, i want it to be a list of words
        summary_title = summary_title.split()
        cluster_title[cluster_id] = summary_title
    cluster_summary_list = [[value] for key, value in cluster_summary.items()]
    cluster_title_list = [[value] for key, value in cluster_title.items()]
    
    
    session["cluster_desc_0"] = cluster_title_list
    
    sessionData = {
        "docs_org": convert_ndarray_to_list(orig_doc_term_mat),  # doc-term raw freq matrix
        "df_org": convert_ndarray_to_list(orig_df),
        "num_cls": num_cls,
        "id2members_0": convert_ndarray_to_list(id2members),
        # "cluster_desc_0": convert_ndarray_to_list(cluster_desc),
        "cluster_desc_0": convert_ndarray_to_list(cluster_title_list),
        "cluster_summary_0": convert_ndarray_to_list(cluster_summary_list),
        "xy_0": convert_ndarray_to_list(cluster_centers),
        "state": state,
        "hue_0": convert_ndarray_to_list(hue),
        "satr_0": convert_ndarray_to_list(satr),
        "org_ids_0": convert_ndarray_to_list(org_ids),
        "dataset": dataset,
        "edges_0": convert_ndarray_to_list(edges),
        "sources": convert_ndarray_to_list(sources)
    }
    
    receive_time = time.clock()
    response_time = str(round(receive_time - send_time, 3))
    

    if entity == "authors" or entity == "experts" or entity == "topics":
        # to tell the difference between keyword search and author search so that sessionStorage won't keep old data when switching between them
        q = q + "_" + entity
    print("state in cluster: ", state)
    print("exit cluster")
    return cluster_summary, cluster_assigned, bibs, cluster_desc, sessionData, response_time

cls_summary, cls_assigned, bibs, cls_desc, sessionData, response_time = cluster_postgres("M. Kirschenbaum", 10)

cluster
PostgreSQL in routes.py
ROWS:
[('Tools for Data-Driven Scholarship: Past, Present, Future', 'On October 22-24, 2008, nearly fifty scholars, librarians, museum professionals, computer scientists, software developers, and funders attended a workshop at Turf Valley Resort in Ellicott City, Maryland, to discuss the past, present, and future of tools that can assist scholarship in an age of massive digital resources. The workshop, co-funded by the National Science Foundation, the National Endowment for the Humanities, and the Institute of Museum and Library Services, brought these people together because of their active engagement with the production and use of these digital tools and their deep knowledge of associated issues such as scholarly communication and sustainability. (Please see Appendix A for a list of attendees.)    The discussion was pragmatic rather than ideological; the goal was to understand from experience and example how such tools could be created, disseminated, a

/Users/aravind/anaconda3/envs/PATTIE_v2/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:1162: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(


Preprocessed Summary:  1. Digital Humanities
2. Exploring digital preservation, scholarly use, and challenges in the evolving digital landscape. 


Cluster 0, Assigned Cluster: '['received', 'authors', 'created', 'part', 'work', 'form', 'read', 'site', 'final', 'version', 'press', 'essay', 'post', 'u201d']'

Summary: Exploring digital preservation, scholarly use, and challenges in the evolving digital landscape.

Title: Digital Humanities


Cluster 1 

 User Content: Clustered Publications:

@article{39, title=4. North of Boston}
@article{40, title=Building a Digital Curation Workstation with BitCurator}
@article{41, title=u201cSo the Colors Cover the Wiresu201d: Interface, Aesthetics, and Usability}'


Preprocessed Summary:  1. Digital Curation Workstation
2. Exploring digital curation tools and interface design for usability in preservation. 


Cluster 1, Assigned Cluster: '['collecting', 'institutions', 'hill', 'carolina', 'sils', 'chapel', 'workflows', 'methods', 'development', 'ef

In [6]:
# Write query to print authors and their frequency
query = """
SELECT authors, COUNT(authors) as author_freq
FROM papers
GROUP BY authors
ORDER BY author_freq DESC;
"""
hostname = '34.133.177.246'
port = 5432
database = 'dcdi'
username = 'aravind'
password = 'C&99Fk6xHxypA2R$C4XQ'
# Execute the query
results, error = execute_query(query, hostname, port, database, username, password)
if error:
    print(error)
else:
    print(results)

[(['M. Phillips'], 195), (['Georg Rehm', 'H. Uszkoreit'], 85), (['Ian Milligan'], 78), (['Georg Rehm', 'Hans Uszkoreit'], 75), (['Chen Liang'], 65), (['N. Fuhr'], 61), (['J. Savoy'], 61), (['D. Bainbridge'], 58), (['J. Swacha'], 43), (['Heiko Paulheim'], 41), (['E. Fox'], 41), (['S. Harding'], 40), (['N. Kando'], 38), (['A. Balsamo'], 38), (['Jimmy J. Lin'], 36), (['J. Schnapp'], 35), (['T. Underwood'], 34), (['M. Kirschenbaum'], 33), (['K. Nørvåg'], 33), (['J. S. Downie'], 31), (['P. Andrews'], 31), (['Steffen Eger'], 29), (['Oren Etzioni'], 29), (['T. McPherson'], 29), (['D. Maier'], 29), (['Timothy Baldwin'], 29), (['C. Hennesy'], 28), (['Jill Walker Rettberg'], 28), (['R. Burke'], 28), (['Philipp Mayr'], 28), (['Sarah-Jane Saravani'], 27), (['P. Juola'], 26), (['N. Ferro'], 26), (['M. Marsili'], 25), (['K. Balog'], 25), (['Lev Manovich'], 25), (['N. Ferro', 'G. Silvello'], 25), (['Steffen Staab'], 25), (['Lars Vogt'], 25), (['G. Bowker'], 24), (['M. Schefczyk'], 24), (['Michael L. 